<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [27]:
# Imports

import requests
import pandas as pd
import numpy as np
import random
import sys
import os

from __future__ import print_function

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

In [2]:
# Get the Shakespeare data with requests

url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]

# Skip the Table of Contents

data = data[135:]

# Fixing Titles

toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 

for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [3]:
#Shakespeare Data Parsed by Play

df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [4]:
# Check shape of dataframe

df_toc.shape

(43, 4)

In [5]:
# Clean the text column

df_toc['text'] = df_toc["text"].str.replace('\n',"")
df_toc['text'] = df_toc["text"].str.replace('\r',"")
df_toc['text'] = df_toc["text"].str.replace('[',"")
df_toc['text'] = df_toc["text"].str.replace(']',"")
df_toc['text'] = df_toc["text"].str.replace('_',"")
df_toc['text'] = df_toc["text"].str.replace('<',"")
df_toc['text'] = df_toc["text"].str.replace('>',"")
df_toc['text'] = df_toc["text"].str.replace('/',"")
df_toc['text'] = df_toc["text"].str.replace('*',"")

In [6]:
# Verify changes in first text

df_toc['text'][0]

'ALL’S WELL THAT ENDS WELLContentsACT IScene I. Rossillon. A room in the Countess’s palace.Scene II. Paris. A room in the King’s palace.Scene III. Rossillon. A Room in the Palace.ACT IIScene I. Paris. A room in the King’s palace.Scene II. Rossillon. A room in the Countess’s palace.Scene III. Paris. The King’s palace.Scene IV. Paris. The King’s palace.Scene V. Another room in the same.ACT IIIScene I. Florence. A room in the Duke’s palace.Scene II. Rossillon. A room in the Countess’s palace.Scene III. Florence. Before the Duke’s palace.Scene IV. Rossillon. A room in the Countess’s palace.Scene V. Without the walls of Florence.Scene VI. Camp before Florence.Scene VII. Florence. A room in the Widow’s house.ACT IVScene I. Without the Florentine camp.Scene II. Florence. A room in the Widow’s house.Scene III. The Florentine camp.Scene IV. Florence. A room in the Widow’s house.Scene V. Rossillon. A room in the Countess’s palace.ACT VScene I. Marseilles. A street.Scene II. Rossillon. The inner 

In [7]:
# Encode Data as Chars

# Gather all text 

# Why? 1. See all possible characters 2. For training / splitting later

text = " ".join(df_toc['text'])

# Unique Characters

chars = list(set(text))

# Lookup Tables

char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [8]:
# Number of unique characters

len(chars)

99

In [9]:
# See integer:characters

int_char

{0: 'ê',
 1: 'Q',
 2: 's',
 3: '1',
 4: 'c',
 5: '-',
 6: '’',
 7: 'Y',
 8: 'k',
 9: '"',
 10: 'â',
 11: 'X',
 12: 'L',
 13: '|',
 14: 'é',
 15: 'z',
 16: 'W',
 17: '0',
 18: 'p',
 19: '.',
 20: 'l',
 21: 'E',
 22: '2',
 23: 'ç',
 24: '7',
 25: 'r',
 26: 'É',
 27: 't',
 28: 'S',
 29: '”',
 30: 'R',
 31: 'B',
 32: 'H',
 33: ')',
 34: 'o',
 35: ';',
 36: 'C',
 37: 'u',
 38: '“',
 39: 'n',
 40: 'P',
 41: 'g',
 42: 'N',
 43: 'æ',
 44: 'j',
 45: '%',
 46: '&',
 47: '3',
 48: '`',
 49: 'd',
 50: '$',
 51: '(',
 52: '!',
 53: 'Æ',
 54: 'm',
 55: 'è',
 56: 'î',
 57: 'y',
 58: 'U',
 59: '8',
 60: 'v',
 61: 'x',
 62: 'O',
 63: "'",
 64: 'A',
 65: '6',
 66: 'œ',
 67: '—',
 68: ':',
 69: 'q',
 70: '\\',
 71: '9',
 72: 'i',
 73: '‘',
 74: '?',
 75: '}',
 76: 'Z',
 77: 'à',
 78: 'f',
 79: '4',
 80: '@',
 81: 'I',
 82: ' ',
 83: ',',
 84: '\t',
 85: 'T',
 86: 'w',
 87: 'a',
 88: 'h',
 89: 'J',
 90: 'D',
 91: 'e',
 92: 'G',
 93: 'V',
 94: 'b',
 95: 'F',
 96: 'M',
 97: '5',
 98: 'K'}

In [10]:
# Create the sequence data

maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1059264


In [11]:
# Check sequence 42

sequences[42]

[27,
 88,
 91,
 82,
 98,
 72,
 39,
 41,
 6,
 2,
 82,
 18,
 87,
 20,
 87,
 4,
 91,
 19,
 28,
 4,
 91,
 39,
 91,
 82,
 81,
 81,
 19,
 82,
 30,
 34,
 2,
 2,
 72,
 20,
 20,
 34,
 39,
 19,
 82,
 64]

In [12]:
 # Check what sequence 0 prints
 
for i in sequences[0]:
    print(int_char[i])

A
L
L
’
S
 
W
E
L
L
 
T
H
A
T
 
E
N
D
S
 
W
E
L
L
C
o
n
t
e
n
t
s
A
C
T
 
I
S
c


In [13]:
# Next char is one element for each sequence, to help the model keep learning

next_char[0], int_char[next_char[0]]

(91, 'e')

In [14]:
# Create X & y

X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i,t,char] = 1
    y[i, next_char[i]] = 1

In [15]:
# See X shape

X.shape

(1059264, 40, 99)

In [16]:
# Array of characters, like a grid for each character, in each word

# ie: no a, b, c or x, y, z in this text, from what is visible.

# Keep in mind the first and last characters could be non-alphabet

X[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [17]:
# See the chars

chars

['ê',
 'Q',
 's',
 '1',
 'c',
 '-',
 '’',
 'Y',
 'k',
 '"',
 'â',
 'X',
 'L',
 '|',
 'é',
 'z',
 'W',
 '0',
 'p',
 '.',
 'l',
 'E',
 '2',
 'ç',
 '7',
 'r',
 'É',
 't',
 'S',
 '”',
 'R',
 'B',
 'H',
 ')',
 'o',
 ';',
 'C',
 'u',
 '“',
 'n',
 'P',
 'g',
 'N',
 'æ',
 'j',
 '%',
 '&',
 '3',
 '`',
 'd',
 '$',
 '(',
 '!',
 'Æ',
 'm',
 'è',
 'î',
 'y',
 'U',
 '8',
 'v',
 'x',
 'O',
 "'",
 'A',
 '6',
 'œ',
 '—',
 ':',
 'q',
 '\\',
 '9',
 'i',
 '‘',
 '?',
 '}',
 'Z',
 'à',
 'f',
 '4',
 '@',
 'I',
 ' ',
 ',',
 '\t',
 'T',
 'w',
 'a',
 'h',
 'J',
 'D',
 'e',
 'G',
 'V',
 'b',
 'F',
 'M',
 '5',
 'K']

In [18]:
# See y shape

y.shape

(1059264, 99)

In [19]:
# Again, 

# maxlen was set above when creating sequence data and 

# the number of unique characters remains the same as before

(maxlen, len(chars))

(40, 99)

In [20]:
# Build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
# Helper function to sample an index from a probability array

def sample(preds):

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [22]:
# Function invoked at end of each epoch. Prints generated text.

def on_epoch_end(epoch, _):
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [29]:
# Fit the model

model.fit(X, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
8271/8276 [============================>.] - ETA: 0s - loss: 2.1664
----- Generating text after Epoch: 0
----- Generating with seed: "er the city, clip your wives, your frien"
er the city, clip your wives, your friend the glino live rideralch as now of conter’ alady bug to the Witnthe, forse peader. Ant ’Wries yoush?  O Pereats, and and your alps leandaPe me thin Hencebid;    A sim your ableares snom gegen hore, Lutelp im treebleve Cefopiest.                             A     gour sure Jarccuk,    Think endonded I many.  BIUS.  Enter,In shanow she moried wo the as there this dejaand Kich fso frolok it lies ou
8276/8276 [==============================] - 54s 6ms/step - loss: 2.1662
Epoch 2/10
8273/8276 [============================>.] - ETA: 0s - loss: 1.9325
----- Generating text after Epoch: 1
----- Generating with seed: " matter. How does your ladyship like it?"
 matter. How does your ladyship like it? I putwer; all made By strouns,Thy verterOr, whol; bost deamine homineta

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN